# Carga de librerías

In [1]:
# Basic
import pandas as pd
import numpy as np
import time
import os
from tqdm import tqdm
import re
# Scraping
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common import exceptions as SeleniumExceptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

# Load web driver

In [2]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [3]:
root = "https://open.spotify.com/collection/tracks"

In [4]:
# Load cookies json
import json
with open('cookies.json', 'r') as f:
    cookies = json.load(f)

In [5]:
for cookie in tqdm(cookies):
  u = driver.execute_cdp_cmd(
                  'Network.setCookie',
                  {
                      'domain': cookie['domain'],
                      'path': cookie['path'],
                      'name': cookie['name'],
                      'value': cookie['value'],
                      'httpOnly': cookie['httpOnly'],
                      'secure': cookie['secure'],
                  },
              )

100%|██████████| 28/28 [00:00<00:00, 195.26it/s]


In [6]:
driver.get(root)

# Start scraping

In [7]:
df_songs = pd.DataFrame(columns=['title', 'artist', 'artist_link', 'artist_description'])

## Scroll until load all songs

In [ ]:
selector_songs_parent = (By.CSS_SELECTOR, "div[data-testid='track-list']")
selector_songs_container = (By.CSS_SELECTOR, "div.JUa6JJNj7R_Y3i4P8YUX")
selector_song_title = (By.CSS_SELECTOR, "a[data-testid='internal-track-link']")
selector_song_artist = (By.CSS_SELECTOR, "a[data-testid='internal-track-link']~span a")
actions = ActionChains(driver)
try:
    # Get the number of songs
    songs_parent = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(selector_songs_parent)
    )
    song_count = songs_parent.get_attribute("aria-rowcount")
    song_count = int(song_count) if song_count else 0
    # Song container to scroll to
    songs_container = songs_parent.find_element(*selector_songs_container)
    body = driver.find_element(By.TAG_NAME, "body")
    for i in tqdm(range(2, song_count + 1)): # Row 1 is the header
      song = None
      for j in range(10): # Scroll until find the song, max 10 tries
        try:
          song = songs_parent.find_element(By.CSS_SELECTOR, f"div[role='row'][aria-rowindex='{i}']")
          break
        except SeleniumExceptions.NoSuchElementException:
          # Use Keys.PAGE_DOWN to scroll
          actions.move_to_element(songs_container).click().send_keys(Keys.PAGE_DOWN).perform()
      if not song:
        raise Exception(f"Song {i} not found")
      song_title = song.find_element(*selector_song_title)
      song_artist = song.find_element(*selector_song_artist)
      song_artist_link = song_artist.get_attribute("href")
      
      df_song = pd.DataFrame({'title': [song_title.text],
                              'artist': [song_artist.text],
                              'artist_link': [song_artist_link],
                              'artist_description': [None]})
      df_songs = pd.concat([df_songs, df_song], ignore_index=True)
      
except SeleniumExceptions.TimeoutException:
    raise Exception("Check if the page is loaded correctly and the CSS selector is correct")

100%|██████████| 262/262 [00:57<00:00,  4.56it/s]


In [12]:
df_songs.shape

(262, 4)